In [1]:
import cobra
from cobra.sampling import sample
from cobra.sampling import OptGPSampler
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import pfba
from cobra.flux_analysis import moma
from cobra.flux_analysis import production_envelope
from cobra.exceptions import OptimizationError, Infeasible
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.patches import Patch
from matplotlib import colors as mcolors
from matplotlib import font_manager
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import mannwhitneyu
from psamm.datasource import native
from psamm.fluxanalysis import FluxBalanceProblem, FluxBalanceError
from psamm.lpsolver import cplex, lp
import seaborn as sns
import os
# Built-in multiprocessing pool module does not work interactively
# Run pip install pathos to use this one
from pathos.multiprocessing import ProcessingPool as Pool 
from functools import partial
from cameo import fba
from collections import defaultdict
from cameo import phenotypic_phase_plane
import pickle
import subprocess
from sklearn.model_selection import ParameterGrid
import matplotlib.ticker as mticker
import lzma
import time

# Simulations for generating modeling results in Bing et al. 2024

Bing, Ryan G., Kathryne C. Ford, Daniel J. Willard, Mohamad J. H. Manesh, Christopher T. Straub, Tunyaboon Laemthong, Benjamin H. Alexander, et al. 2024. “Engineering Ethanologenicity into the Extremely Thermophilic Bacterium Anaerocellum (f. Caldicellulosiriuptor) Bescii.” Metabolic Engineering, September. https://doi.org/10.1016/j.ymben.2024.09.007.

Simulations were originally run on commit 5a6b7043a6380469c54656b7cb1d71263dc33f1a. This version of the model is saved as an sbml file in "GEM-iCbes/additional_files/5a6b7043.sbml". All simulations in this notebook reference the sbml file and will work the same regardless of which commit is currently loaded.

In [2]:
def create_rxn(model, name, cpd_dict, direction="both", limits=None):
    # Create a reaction from a dict with {"CPD1": -1, "CPD2: 1"}
    # Assuming the cpd ids are already in the model
    if direction not in ["forward", "reverse", "both"]:
        raise ValueError("Direction must be either forward, reverse, or both")
    rxn = cobra.Reaction(name)
    if limits is not None:
        if len(limits) != 2:
            raise ValueError("Limits must be of length 2")
        else:
            rxn.lower_bound = limits[0]
            rxn.upper_bound = limits[1]
    elif direction == "both":
        rxn.lower_bound = -1000
        rxn.upper_bound = 1000
    elif direction == "forward":
        rxn.lower_bound = 0
        rxn.upper_bound = 1000
    else:
        rxn.lower_bound = -1000
        rxn.upper_bound = 0
    rxn.add_metabolites({
        getattr(model.metabolites, cpd): val for cpd, val in cpd_dict.items()
    })
    print("Adding the following reaction to model:")
    print(rxn)
    model.add_reactions([rxn])

## Jun 26, 2024 - Final version

For the paper, we might want to just pick specific time points rather than using the min and max. I am making some code to try out all of these options.

In [8]:
### Model setup
with open("../additional_files/5a6b7043.sbml", "r") as f:
    cbes_model = cobra.io.read_sbml_model(f)

create_rxn(cbes_model, "gly_sink", {"C00037": -1}, direction="forward")
create_rxn(cbes_model, "glu_sink", {"C00025": -1}, direction="forward")
create_rxn(cbes_model, "ile_sink", {"C00407": -1}, direction="forward")
create_rxn(cbes_model, "leu_sink", {"C00123": -1}, direction="forward")
create_rxn(cbes_model, "pro_sink", {"protein": -1}, direction="forward")
create_rxn(cbes_model, "met_sink", {"C00073": -1}, direction="forward")

cbes_model.reactions.EX_C00760_kz_LSQBKT_e_RSQBKT_.lower_bound = -0.979228
cbes_model.reactions.EX_C00031_LSQBKT_e_RSQBKT_.lower_bound = 0

# Creating a map for cpd names to target rxns to constrain them
media_rxns = [r for r in cbes_model.reactions if (r.id.startswith("EX_") or ("sink" in r.id))]
media_cpds = [list(rxn.metabolites.keys())[0].id for rxn in media_rxns]
media_rxn_names = [r.id for r in media_rxns]

media_info_df = pd.DataFrame([media_cpds, media_rxn_names],
                             index=["cpd", "rxn"],
                             columns=[getattr(cbes_model.metabolites, x).name for x in media_cpds]
                             ).T.sort_index()

media_info_df.loc["Protein", "rxn"] = "pro_met"

# Modifications to the media (2 mM phosphate, 2x vitamins)
# 2x vitamins
vitamins = ["Nicotinate", "Pantothenate", "Lipoate", "4-Aminobenzoate",
            "Thiamine", "Riboflavin", "Pyridoxine", "Vitamin B12", "Biotin", "Folate"]
for rxn in media_info_df.loc[vitamins, "rxn"]:
    getattr(cbes_model.reactions, rxn).lower_bound *= 2
    
# 2 mM phosphate
getattr(cbes_model.reactions, media_info_df.loc["Orthophosphate", "rxn"]).lower_bound = -2

# Free minerals
getattr(cbes_model.reactions, media_info_df.loc["Zinc cation", "rxn"]).lower_bound = -1000
getattr(cbes_model.reactions, media_info_df.loc["Fe2+", "rxn"]).lower_bound = -1000

# Giving it some uracil
getattr(cbes_model.reactions, media_info_df.loc["Uracil", "rxn"]).lower_bound = -1

# Add hco3, extracellular spontaneous ionization of HCO3- to CO2
create_rxn(cbes_model, "hco3_e",
           {"C00011_1": -1, "C00001": -1, "C00288_1": 1, "C00080_1": 1},
           direction="reverse")

# Alter the second AdhE reaction to use NADPH or NADH
create_rxn(cbes_model, "R00754", # NADPH  version
           {"C00084": -1, "C00004": -1, "C00080": -1, "C00469": 1, "C00003": 1}, direction="forward")
create_rxn(cbes_model, "R00746",  # NADH version
           {"C00084": -1, "C00005": -1, "C00080": -1, "C00469": 1, "C00006": 1}, direction="forward")
create_rxn(cbes_model, "R00228",
           {"C00024": -1, "C00004": -1, "C00080": -1, "C00084": 1, "C00010": 1, "C00003": 1})

bicarb_model = cbes_model.copy()
hydroxide_model = cbes_model.copy()

### CALCULATE THE 9% BICARB ADDITION?
media_info_df.loc["HCO3-"]
getattr(bicarb_model.reactions, media_info_df.loc["HCO3-", "rxn"]).lower_bound = -2.6*1000/61.0168
getattr(hydroxide_model.reactions, media_info_df.loc["HCO3-", "rxn"]).lower_bound = -0.1*1000/61.0168

## Set CO2 reactions to be reversible
#for rxn in cbes_model.metabolites.C00011.reactions:
#    if (rxn.reversibility is False):
#        rxn.lower_bound = -1000
#        rxn.upper_bound = 1000

'' is not a valid SBML 'SId'.


Adding the following reaction to model:
gly_sink: C00037 --> 
Adding the following reaction to model:
glu_sink: C00025 --> 
Adding the following reaction to model:
ile_sink: C00407 --> 
Adding the following reaction to model:
leu_sink: C00123 --> 
Adding the following reaction to model:
pro_sink: protein --> 
Adding the following reaction to model:
met_sink: C00073 --> 
Adding the following reaction to model:
hco3_e: C00001 + C00011_1 <-- C00080_1 + C00288_1
Adding the following reaction to model:
R00754: C00004 + C00080 + C00084 --> C00003 + C00469
Adding the following reaction to model:
R00746: C00005 + C00080 + C00084 --> C00006 + C00469
Adding the following reaction to model:
R00228: C00004 + C00024 + C00080 <=> C00003 + C00010 + C00084


In [9]:
### Calculating differences for all products
df = pd.read_csv("../additional_files/RKCB92_Avicel_Bicarb_Hydroxide_RXTRS_for_URI_20240428.xlsx - Copy of Soluble Product Summary.csv",
           header=[0,1])
# Formatting the table
df = df.loc[:, df.apply(lambda x: not all(x.isna()), axis=0)]
prev = ""
newcols = []
for x in df.columns:
    if not x[0].startswith("Unnamed"):
        prev = x[0]
    newcols.append((prev, x[1]))
df.columns = pd.MultiIndex.from_tuples(newcols)
newvals = []
for i in range(len((tmplist := df.loc[:,("", "Time point (hr)")]))):
    if tmplist[i] == "stdev":
        newvals.append((tmplist[i-1], "stdev"))
    else:
        newvals.append((tmplist[i], "mean"))
df = df.drop(columns=("", "Time point (hr)")).set_index(pd.MultiIndex.from_tuples(newvals))

# Separate means and stdevs since it's easier later
df2 = (
    df.drop(columns=("Notes", "Reactors ran until ethanol titers started to drop"))
    .replace(to_replace=np.nan, value=0)
    .replace(to_replace="#DIV/0!", value=0)
)
df2_means = (
    df2.loc[(slice(None), "mean"), :]
    .droplevel(1).T
    .reset_index(level=1)
    .rename(columns={"level_0": "condition", "level_1": "Time (hrs)"})
    .drop(columns="Cellooligosacchides detecetd (yes/no)")
    .astype(float)
)
df2_stdevs = (
    df2.loc[(slice(None), "stdev"), :]
    .droplevel(1).T
    .reset_index(level=1)
    .rename(columns={"level_0": "condition", "level_1": "Time (hrs)"})
    .astype(float)
)

# Calculate changes for each measured compound from the table using different schemes
diffs_dict = {
    "Last-First": df2_means.groupby(df2_means.index).apply(lambda x: x.iloc[-1] - x.iloc[0]),
    "2ndLast-First": df2_means.groupby(df2_means.index).apply(lambda x: x.iloc[-2] - x.iloc[0]),
    "3rdLast-First": df2_means.groupby(df2_means.index).apply(lambda x: x.iloc[-3] - x.iloc[0]),
    "Max-Min": df2_means.groupby(df2_means.index).apply(lambda x: x.max() - x.min())
}

# Adding the avicel and cell dry weight numbers which were measured separately
avicel_df = pd.read_csv("../additional_files/RKCB92_Avicel_Bicarb_Hydroxide_RXTRS_for_URI_20240428.xlsx - Avicel and Cell Masses.csv")
more_diffs = (avicel_df.loc[
    avicel_df["Unnamed: 0"].isin(["Scaled Avicel consumed (g/L)", "Dry Cell Weight (g/L-culture)"]),
    ["Unnamed: 0", "RKCB92, sodium bicarbonate condition", "RKCB92, sodium hydroxide condition"]]
    .set_index("Unnamed: 0")
    .T
)

all_diffs = {key: diffs.join(more_diffs) for key, diffs in diffs_dict.items()}

In [10]:
### Create constraints df from the differences
# Maps the names from Ryan's table to their names in the model
# And gives a scaling factor that converts to exchange constraint values in mM
measurement_map = pd.DataFrame({
#    "Uracil": ("Uracil", -1/1000),
    "NH4 Consumed (mM)": ("NH4+", -1),
    "Cellobiose (g/L)": ("Cellobiose", 1000/342.3),
    "Glucose (g/L)": ("D-Glucose", 1000/180.156),
    "Acetate (mM)": ("Acetate", 1),
    "Acetoin": ("Acetoin", 1),
    "Ethanol": ("Ethanol", 1),
    "Pyruvate": ("Pyruvate", 1),
    "Glutamate": ("L-Glutamate", 1),                              # model is missing sink for this: C00025
    "Glycine": ("Glycine", 1),                                    # model is missing sink for this: C00037
    "Alanine": ("L-Alanine", 1),
    "Valine": ("L-Valine", 1),
    "Isoleucine": ("L-Isoleucine", 1),                            # model is missing sink for this: C00407
    "Leucine": ("L-Leucine", 1),                                  # model is missing sink for this: C00123
    "Total Protein (mg/ml)": ("Protein", 1),
#    "pH", ("H+", -1),                                            # ignoring pH
    "Dry Cell Weight (g/L-culture)": ("Biomass", 1),
    "Scaled Avicel consumed (g/L)": ("Cellulose(n=218)", -1000/35364.58)
}, index=["model_name", "scale"]).T

constraint_dfs = {}
for key, diffs in all_diffs.items():
    tmp = diffs.T.join(measurement_map, how="right")
    tmp["bicarb_constraint"] = tmp["RKCB92, sodium bicarbonate condition"].astype(float) * tmp["scale"]
    tmp["hydroxide_constraint"] = tmp["RKCB92, sodium hydroxide condition"].astype(float) * tmp["scale"]
    constraint_dfs[key] = tmp.join(media_info_df, on="model_name")[["model_name", "bicarb_constraint", "hydroxide_constraint", "rxn"]]

In [11]:
results_tmp = []
for key, constraint_df in constraint_dfs.items():
    # We force everything other than Cellulose to be EXACTLY as specified in the table (fixed upper and lower bound)
    # Cellulose only has lower bound fixed, so it can consume UP TO the measured amount of carbon
    #print(f"Simulating {key} condition")
    
    bicarb_tmp = bicarb_model.copy()
    hydroxide_tmp = hydroxide_model.copy()
    # Set constraints
    for x, row in constraint_df.iterrows():
        #print(f"constraining {row['model_name']}")
        getattr(bicarb_tmp.reactions, media_info_df.loc[row["model_name"], "rxn"]).lower_bound = row["bicarb_constraint"]
        getattr(hydroxide_tmp.reactions, media_info_df.loc[row["model_name"], "rxn"]).lower_bound = row["hydroxide_constraint"]

        if row["model_name"] not in ["Cellulose(n=218)", "NH4+"]: 
            getattr(bicarb_tmp.reactions, media_info_df.loc[row["model_name"], "rxn"]).upper_bound = row["bicarb_constraint"]
            getattr(hydroxide_tmp.reactions, media_info_df.loc[row["model_name"], "rxn"]).upper_bound = row["hydroxide_constraint"]
    
    try:
        bicarb_result = flux_variability_analysis(bicarb_tmp, ["EX_C00760_kz_LSQBKT_e_RSQBKT_", "EX_C00097_LSQBKT_e_RSQBKT_"])
        hydroxide_result = flux_variability_analysis(hydroxide_tmp, ["EX_C00760_kz_LSQBKT_e_RSQBKT_", "EX_C00097_LSQBKT_e_RSQBKT_"])
        tmp_df = pd.DataFrame([
            bicarb_result.loc["EX_C00760_kz_LSQBKT_e_RSQBKT_", "minimum"],
            hydroxide_result.loc["EX_C00760_kz_LSQBKT_e_RSQBKT_", "minimum"],
            bicarb_result.loc["EX_C00760_kz_LSQBKT_e_RSQBKT_", "minimum"]/constraint_df.loc["Scaled Avicel consumed (g/L)", "bicarb_constraint"],
            hydroxide_result.loc["EX_C00760_kz_LSQBKT_e_RSQBKT_", "minimum"]/constraint_df.loc["Scaled Avicel consumed (g/L)", "hydroxide_constraint"],
            bicarb_result.loc["EX_C00097_LSQBKT_e_RSQBKT_", "maximum"],
            hydroxide_result.loc["EX_C00097_LSQBKT_e_RSQBKT_", "maximum"]
        ], index=["Carbonate model max flux", "Hydroxide model max flux", "Carbonate closure", "Hydroxide closure", 
                  "Carbonate model min cysteine", "Hydroxide model min cysteine"], columns=[key])
        results_tmp.append(tmp_df)
    except Infeasible as e:
        print("Infeasible")
        pass

results_df = pd.concat(results_tmp, axis=1)

In [12]:
results_df

,Last-First,2ndLast-First,3rdLast-First,Max-Min
Carbonate model max flux,-0.693180,-0.660906,-0.614531,-0.705958
Hydroxide model max flux,-0.358653,-0.326850,-0.193901,-0.368926
Carbonate closure,0.707884,0.674925,0.627567,0.720933
Hydroxide closure,0.723537,0.659378,0.391172,0.744262
Carbonate model min cysteine,-0.599303,-0.630823,-0.521241,-0.632970
Hydroxide model min cysteine,-0.377871,-0.352210,-0.218600,-0.378300
